In [17]:
import tkinter as tk
import customtkinter as ctk
from tkinter import messagebox, filedialog
from RunLength import RunLengthEncoder
from Golomb import GolombEncoder
from Arithmetic import ArithmeticEncoder
from LZW import LZWEncoder
from Huffman import HuffmanEncoder
from collections import OrderedDict

In [18]:
class TextEncodingApp(ctk.CTk):
    def __init__(self):
        super().__init__()

        # window properties
        self.title("Compression Techniques")
        self.iconbitmap("images/icon.ico")
        ctk.set_appearance_mode("Light")
        ctk.set_default_color_theme("green")

        # Make the App start on the center of the screen
        screen_width = self.winfo_screenwidth()
        screen_height = self.winfo_screenheight()
        self.x = (screen_width / 2) - (1280 / 2)
        self.y = (screen_height / 2) - (720 / 2)
        self.geometry(f"1280x720+{int(self.x)}+{int(self.y)}")

        self.title_label = ctk.CTkLabel(self, text="Text Compression", text_color="#6527BE", font=("Arial Bold", 24))
        self.title_label.pack(pady=5)

        self.theme_button = ctk.CTkButton(self, text="Change Theme", corner_radius=5, command=self.change_theme)
        self.theme_button.pack(side="top", anchor="ne", padx=10)

        self.input_text = ctk.CTkTextbox(self, width=screen_width/4, height=100)
        self.input_text.pack(pady = 10)

        buttons_frame = ctk.CTkFrame(self, fg_color="transparent")
        buttons_frame.pack(pady=10)

        self.encode_all_button = ctk.CTkButton(buttons_frame, text="Encode All", corner_radius=5, command=self.encode_all)
        self.encode_all_button.pack(side="left", padx=(10, 0), pady=5)

        self.or_label = ctk.CTkLabel(buttons_frame, text="OR", font=("Arial", 12, "bold"))
        self.or_label.pack(side="left", padx=(10, 10), pady=5)

        self.file_picker_button = ctk.CTkButton(buttons_frame, text="Browse a File", corner_radius=5, command=self.read_file)
        self.file_picker_button.pack(side="left", padx=(0, 10), pady=5)
    
        frames = ctk.CTkFrame(self, fg_color="transparent")
        frames.pack(fill="both", expand=True, pady=10)

        self.run_length_frame = ctk.CTkScrollableFrame(frames, corner_radius=12, fg_color="#81b29a")
        self.run_length_frame.pack(anchor='n', side='left', fill="both", expand=True, pady=10, padx=15)
        ctk.CTkButton(self.run_length_frame, text="Encode", width=20, corner_radius=5, command=lambda: self.encode_one(self.run_length_frame)).pack(pady=120)

        self.huffman_frame = ctk.CTkScrollableFrame(frames, corner_radius=12, fg_color="#c9ada7")
        self.huffman_frame.pack(anchor='n', side='left', fill="both", expand=True, pady=10, padx=15)
        ctk.CTkButton(self.huffman_frame, text="Encode", width=20, corner_radius=5, command=lambda: self.encode_one(self.huffman_frame)).pack(pady=120)

        self.LZW_frame = ctk.CTkScrollableFrame(frames, corner_radius=12, fg_color="#7D5260")
        self.LZW_frame.pack(anchor='n', side='left', fill="both", expand=True, pady=10, padx=15)
        ctk.CTkButton(self.LZW_frame, text="Encode", width=20, corner_radius=5, command=lambda: self.encode_one(self.LZW_frame)).pack(pady=120)

        self.golomb_frame = ctk.CTkScrollableFrame(frames, corner_radius=12, fg_color="#2a9d8f")
        self.golomb_frame.pack(anchor='n', side='left', fill="both", expand=True, pady=10, padx=15)
        ctk.CTkButton(self.golomb_frame, text="Encode", width=20, corner_radius=5, command=lambda: self.encode_one(self.golomb_frame)).pack(pady=120)

        self.arithmetic_frame = ctk.CTkScrollableFrame(frames, corner_radius=12, fg_color="#915175")
        self.arithmetic_frame.pack(anchor='n', side='left', fill="both", expand=True, pady=10, padx=15)
        ctk.CTkButton(self.arithmetic_frame, text="Encode", width=20, corner_radius=5, command=lambda: self.encode_one(self.arithmetic_frame)).pack(pady=120)


        self.recommendation_frame = ctk.CTkFrame(self, fg_color="#A6CF98", height=130)
        self.recommendation_frame.pack(fill='x', padx=15, pady=10)

    def update_frame(self, frame, frame_title, encoded_text, bits_before, bits_after, compression_ratio, entropy, average_length, efficiency, probability):
        
        label_frame = ctk.CTkFrame(frame, fg_color="transparent")
        label_frame.pack()

        ctk.CTkLabel(label_frame, text=frame_title, font=("Arial", 20, "bold")).pack(side="left", pady=5)

        encode_button = ctk.CTkButton(label_frame, text="Encode", width=20, corner_radius=5, command=lambda: self.encode_one(frame))
        encode_button.pack(side="left", padx=10, pady=5)


        text_box = ctk.CTkTextbox(frame, height=70)
        text_box.delete('0.0', 'end')
        text_box.insert('0.0', encoded_text)
        text_box.configure(state="disabled")
        text_box.pack(fill="both", expand=True, padx=10, pady=5)

        bits_before_label = ctk.CTkLabel(frame, text=f'bits(before): {bits_before}', font=("Arial", 12, "bold"))
        bits_before_label.pack(anchor='w', padx=10, pady=5)

        bits_after_label = ctk.CTkLabel(frame, text=f'bits(after): {bits_after}', font=("Arial", 12, "bold"))
        bits_after_label.pack(anchor='w', padx=10, pady=5)

        entropy_label = ctk.CTkLabel(frame, text=f'Entropy: {entropy}', font=("Arial", 12, "bold"))
        entropy_label.pack(anchor='w', padx=10, pady=5)

        average_length_label = ctk.CTkLabel(frame, text=f'Average Length: {average_length}', font=("Arial", 12, "bold"))
        average_length_label.pack(anchor='w', padx=10, pady=5)

        compression_ratio_label = ctk.CTkLabel(frame, text=f'Compression Ratio: {compression_ratio}%', font=("Arial", 12, "bold"))
        compression_ratio_label.pack(anchor='w', padx=10, pady=5)

        efficiency_label = ctk.CTkLabel(frame, text=f'Efficiency: {efficiency}%', font=("Arial", 12, "bold"))
        efficiency_label.pack(anchor='w', padx=10, pady=5)
        
        probability_label = ctk.CTkLabel(frame, text='Probabilities: ', font=("Arial", 12, "bold"))
        probability_label.pack(anchor='w', padx=10, pady=5)

        for char, probability in probability.items():
            if char == ' ':
                char = 'Space'
            if char == '\n':
                char = 'New Line'
            ctk.CTkLabel(frame, text=f'{char}: {round(probability, 2)}', font=("Arial", 12)).pack(anchor='w', padx=30)

    def read_file(self):
        file_path = filedialog.askopenfilename(initialdir=r"E:\Faculty Material\Third Level\Second Term\Data Compression\Final Project", title="Select A File", filetypes=(("Text Files", "*.txt"),("all files", "*.*")))
        if not file_path:
            messagebox.showerror("File Error", "No file selected.")
            return
        try:
            text = open(file_path, 'r').read()
            
        except Exception as e:
            messagebox.showerror("File Error", f"Error reading file")
            return
        
        if not text:
            messagebox.showerror("Empty File", "The selected file is empty.")
            return
        
        self.input_text.delete('0.0', 'end-1c')
        self.input_text.insert('0.0', text)
    

    def encode_one(self, frame):
        text = self.input_text.get('0.0', 'end-1c')
        if not text:
            messagebox.showerror("Empty Text", "Please enter a text to encode.")
            return
        
        if frame == self.run_length_frame:
            label = 'Run-Length'
            encoder = RunLengthEncoder.RLE_encoding(text)

        elif frame == self.huffman_frame:
            label = 'Huffman'
            Huffman = HuffmanEncoder(text)
            encoder = Huffman.results()

        elif frame == self.arithmetic_frame:
            label = 'Arithmetic'
            self.frequency_table = None
            self.show_frequency_window(text)
            if not self.frequency_table:
                messagebox.showerror("Frequency Table Error", "Please enter the frequency table.")
                return
            AE = ArithmeticEncoder(self.frequency_table)
            encoder = AE.encode(text, probability_table=AE.probability_table)
    
        elif frame == self.golomb_frame:
            label = 'Golomb'
            order_dialog = ctk.CTkInputDialog(text="Enter the order for Golomb encoding:", title="Golomb Order")
            order = order_dialog.get_input()
            if not order:
                return
            try:
                order = int(order)
            except ValueError:
                messagebox.showerror("Input Error", "Invalid input format for Golomb order.")
                return
            
            encoder = GolombEncoder.golomb_encode(text, order)

        elif frame == self.LZW_frame:
            label = 'LZW'
            lzw = LZWEncoder(text)
            encoder = lzw.get_results()


        # Clear the frame
        for widget in frame.winfo_children():
            widget.destroy()

        # Clear the recommendation frame
        for widget in self.recommendation_frame.winfo_children():
            widget.destroy()

        # Update the frame
        self.update_frame(frame, label, encoder["encoded_text"], encoder["bits_before"], encoder["bits_after"], encoder["compression ratio (%)"], encoder["entropy"],
                        encoder["average_length"], encoder["efficiency"], encoder['probabilities'])
        
        return encoder["compression ratio (%)"]
      
    
    def show_frequency_window(self, text):
        # Create a popup window for frequency table input
        freq_input_window = ctk.CTkToplevel(self)
        # window properties
        freq_input_window.title("Frequency Table")
        freq_input_window.iconbitmap("images/icon.ico")
        freq_input_window.resizable(False, False)
        freq_input_window.wm_attributes('-topmost', 1)
        
        # Label and text entry fields for character and frequency input
        ctk.CTkLabel(freq_input_window, text="Character").grid(row=0, column=0)
        ctk.CTkLabel(freq_input_window, text="Frequency").grid(row=0, column=1)
        
        char_entries = []
        freq_entries = []
        
        unique_chars = list(OrderedDict.fromkeys(text))
        unique_chars.sort()
        
        for i, char in enumerate(unique_chars):
            ctk.CTkLabel(freq_input_window, text=char).grid(row=i+1, column=0)
            char_entry = ctk.CTkEntry(freq_input_window)
            char_entry.insert(0, char)
            char_entry.configure(state='readonly')
            char_entry.grid(row=i+1, column=0, padx=5, pady=5)
            freq_entry = ctk.CTkEntry(freq_input_window)
            freq_entry.grid(row=i+1, column=1, padx=5, pady=5)
            char_entries.append(char_entry)
            freq_entries.append(freq_entry)
     
        # Button to confirm the frequency table
        confirm_button = ctk.CTkButton(freq_input_window, text="Confirm", command=lambda: self.store_frequency(char_entries, freq_entries, freq_input_window))
        confirm_button.grid(row=len(unique_chars) + 1, column=0, columnspan=2, pady=10)

        self.wait_window(freq_input_window)

    def store_frequency(self, char_entries, freq_entries, window):
        frequency_table = {}
        for i, char_entry in enumerate(char_entries):
            char = char_entry.get()
            freq = freq_entries[i].get()
            try:
                freq = float(freq)
            except:
                window.destroy()
                messagebox.showerror("Invalid Input", "Invalid input format for frequency.")
                return
            frequency_table[char] = freq
        self.frequency_table = frequency_table
        window.destroy()

    def encode_all(self):
        text = self.input_text.get('1.0', 'end-1c')
        if not text:
            messagebox.showerror("Empty Text", "Please enter a text to encode.")
            return
    
        frames = [self.run_length_frame, self.huffman_frame, self.LZW_frame, self.golomb_frame, self.arithmetic_frame]
        
        compression_ratios = []

        for frame in frames:
            cr = self.encode_one(frame)
            compression_ratios.append(cr)

        self.update_recommendations(compression_ratios)
            
    def update_recommendations(self, compression_ratios):
        max_cr = max(compression_ratios)
        max_index = compression_ratios.index(max_cr)
        labels = ['Run-Length', 'Huffman', 'LZW', 'Golomb', 'Arithmetic']

        for widget in self.recommendation_frame.winfo_children():
            widget.destroy()


        ctk.CTkLabel(self.recommendation_frame, text='Recommendation', text_color="#6527BE", font=("Arial Bold", 18)).pack(pady=10)
        self.recommendation_label = ctk.CTkLabel(self.recommendation_frame, text=f'The best compression technique for this text is: {labels[max_index]}', font=("Arial Bold", 12))
        self.recommendation_label.pack(anchor="center", pady=10)

        self.recommendation_label = ctk.CTkLabel(self.recommendation_frame, text=f'With Compression Ratio: {max_cr}%', font=("Arial Bold", 12))
        self.recommendation_label.pack(anchor="center", pady=10)
        

    def change_theme(self):
        if ctk.get_appearance_mode() == "Light":
            ctk.set_appearance_mode("Dark")
        else:
            ctk.set_appearance_mode("Light")

        
if __name__ == "__main__":
    app = TextEncodingApp()
    app.mainloop()
